In [7]:
# パッケージのimport
import torch.nn as nn
import torch.nn.init as init
import torch.nn.functional as F
from torch.autograd import Function
import torch.utils.data as data
import torch
import cv2
import numpy as np
import os.path as osp
import glob, pprint
from itertools import product as product
from math import sqrt as sqrt

# XMLをファイルやテキストから読み込んだり、加工したり、保存したりするためのライブラリ
import xml.etree.ElementTree as ET

# フォルダ「utils」のdata_augumentation.pyからimport。入力画像の前処理をするクラス
from utils.data_augumentation import Compose, ConvertFromInts, ToAbsoluteCoords, PhotometricDistort, Expand, RandomSampleCrop, RandomMirror, ToPercentCoords, Resize, SubtractMeans

# フォルダ「utils」にある関数matchを記述したmatch.pyからimport
from utils.match import match

In [8]:
rootpath = "./data/JPMathDataset/train/"

png_path_template = osp.join(rootpath + "img/" + "*.png")
xml_path_template = osp.join(rootpath + "xml/" + "*.xml")

img_list = glob.glob(png_path_template)
anno_list = glob.glob(xml_path_template)

print(len(img_list))
print(len(anno_list))

for i in range(len(img_list)):
    img = img_list[i]
    img_num = img[-11:-4]
    anno = anno_list[i]
    anno_num = anno[-11:-4]
    if img_num != anno_num:
        print(img_num)


1723
1723


In [12]:
rootpath = "./data/JPMathDataset/"

png_path_template = osp.join(rootpath + "img/" + "*.png")
xml_path_template = osp.join(rootpath + "xml/" + "*.xml")

img_list = glob.glob(png_path_template)
anno_list = glob.glob(xml_path_template)

valid_index = []

for i in range(len(anno_list)):

    xml_path = anno_list[i]
    xml = ET.parse(xml_path).getroot()

    if xml.find('object') != None:
        valid_index.append(i)

valid_img_list = []
valid_anno_list =  []
for j in valid_index:
    valid_img_list.append(img_list[j])
    valid_anno_list.append(anno_list[j])

print(valid_img_list)
print(valid_anno_list)


['./data/JPMathDataset/img\\000_000.png', './data/JPMathDataset/img\\000_001.png', './data/JPMathDataset/img\\000_002.png', './data/JPMathDataset/img\\000_003.png', './data/JPMathDataset/img\\000_004.png', './data/JPMathDataset/img\\000_005.png', './data/JPMathDataset/img\\000_006.png', './data/JPMathDataset/img\\000_007.png', './data/JPMathDataset/img\\000_008.png', './data/JPMathDataset/img\\000_009.png', './data/JPMathDataset/img\\000_010.png', './data/JPMathDataset/img\\000_011.png', './data/JPMathDataset/img\\000_013.png']
['./data/JPMathDataset/xml\\000_000.xml', './data/JPMathDataset/xml\\000_001.xml', './data/JPMathDataset/xml\\000_002.xml', './data/JPMathDataset/xml\\000_003.xml', './data/JPMathDataset/xml\\000_004.xml', './data/JPMathDataset/xml\\000_005.xml', './data/JPMathDataset/xml\\000_006.xml', './data/JPMathDataset/xml\\000_007.xml', './data/JPMathDataset/xml\\000_008.xml', './data/JPMathDataset/xml\\000_009.xml', './data/JPMathDataset/xml\\000_010.xml', './data/JPMath

In [13]:
# 「XML形式のアノテーション」を、リスト形式に変換するクラス

ind = 1

img = cv2.imread(img_list[ind])  # [高さ][幅][色BGR]
height, width, channels = img.shape

txt_path = anno_list[ind]

# 画像内の全ての物体のアノテーションをこのリストに格納します
ret = []

# txtファイルを読み込む
with open(txt_path, mode='rt') as f:
    anno_data = f.readlines()

for i in range(4, len(anno_data)):
    bndbox = anno_data[i].replace(' ', '').replace('\n', '').split("\t")
    for j in range(len(bndbox)):
        bndbox[j] = float(bndbox[j])

    # アノテーションの xmin, ymin, xmax, ymaxを取得し、0～1に規格化
    bbox_xywh = bndbox[:4]
    xmin = bbox_xywh[0]
    ymin = bbox_xywh[1]
    xmax = xmin + bbox_xywh[2]
    ymax = ymin + bbox_xywh[3]

    # アノテーションのクラスを追加(Inline:0, Display:1)
    bndbox = [xmin/100, ymin/100, xmax/100, ymax/100, int(bndbox[4])]

    ret += [bndbox]

pprint.pprint(ret)


ValueError: could not convert string to float: ''

In [1]:
a = [[] for i in range(5)]
print(a)

[[], [], [], [], []]
